In [2]:
#| default_exp psql

In [3]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'..')

In [4]:
#| export
from dotenv import load_dotenv
import os


from sqlalchemy import create_engine, MetaData, inspect, Column, ForeignKey, JSON
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker,mapped_column
from sqlalchemy.sql import text
from sqlalchemy.dialects.postgresql import UUID
from pgvector.sqlalchemy import Vector
from sqlalchemy import Column, Integer, String ,DateTime,UniqueConstraint,Table, BIGINT

import uuid

In [5]:
#| export
def query2df(query):
    query_str = query.statement.compile(dialect=query.session.bind.dialect)
    return pd.read_sql(query_str, query.session.bind)

In [6]:
#| export

load_dotenv('/app/.env')
POSTGRES_USER = os.getenv('POSTGRES_USER')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')



DATABASE_URL = f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@db:5432/db"
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()

/tmp/ipykernel_5835/2249259410.py:13: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [7]:
inspector = inspect(engine)

In [8]:
#| export
schema = 'speakers'
LocalBase = declarative_base(metadata=MetaData(schema=schema))



def engine_execute(statement):
    with engine.begin() as connection:
        connection.execute(text(statement))

/tmp/ipykernel_5835/3982815251.py:3: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  LocalBase = declarative_base(metadata=MetaData(schema=schema))


In [9]:
#engine_execute('CREATE EXTENSION vector;')

In [10]:
#| export

class SpeakerEmbs(LocalBase):
    __tablename__ = 'speakerembs'
    id = mapped_column(UUID(as_uuid=True), primary_key=True, default=uuid.uuid4)
    embedding = mapped_column(Vector(256))
    speaker_id = mapped_column(UUID(as_uuid=True), ForeignKey('speakers.id'), nullable=False)
    client_id = Column(UUID(as_uuid=True), nullable=False)



class Speakers(LocalBase):
    __tablename__ = 'speakers'
    id = mapped_column(UUID(as_uuid=True), primary_key=True, default=uuid.uuid4)
    client_id = Column(UUID(as_uuid=True), nullable=False)

In [11]:
#engine_execute(f'drop table speakers.speakers;')

In [12]:
engine_execute(f'CREATE SCHEMA IF NOT EXISTS {schema};')
LocalBase.metadata.create_all(engine)

In [13]:
import pandas as pd

/tmp/ipykernel_5835/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [18]:
pd.read_sql('select * from speakers.speakerembs',engine)

,id,embedding,speaker_id,client_id
0,f86608fb-c600-457a-802a-08b8cbe49bfc,"[-0.0060930643,0.24502955,-0.017237157,-0.0625...",9da16d6d-5710-44e7-aa1d-ce9cb7221b29,851f343e-4954-4f0a-8835-9664cc91c181
1,8b2372dd-6c80-44d6-886d-49a54602c7b8,"[-0.22509952,0.033845965,0.25711492,0.09201576...",357134c2-4e0e-4e1e-a503-e95c851a1953,851f343e-4954-4f0a-8835-9664cc91c181
2,fe1dfda8-9ccf-4d8e-a255-ae9c5f087bb9,"[0.094240665,0.22188322,0.054099936,-0.0152598...",81ddc4cf-0fa2-4702-8f1e-c1eedb2a701c,851f343e-4954-4f0a-8835-9664cc91c181
3,3ae1a14e-5178-4265-bb92-fc95f8255293,"[-0.027464962,0.0046889866,0.06964608,-0.03182...",079cc4b7-8060-4545-b2cc-bc118b86b655,851f343e-4954-4f0a-8835-9664cc91c181
4,5c6acd0f-8c23-463b-beeb-03132a2737c7,"[-0.10180586,0.19403334,0.111121945,0.02353122...",1e64ed21-1aac-4404-aa5a-c8f5cd13fa9c,851f343e-4954-4f0a-8835-9664cc91c181
5,66872e3f-6986-4c6f-a34c-69296b643be7,"[0.11575112,-0.031971682,0.09947285,0.03251374...",960931bd-28ef-47d2-b5f9-da652b41db49,851f343e-4954-4f0a-8835-9664cc91c181
6,0267d6bb-9db7-4e60-9a22-43831e070479,"[0.017613009,0.12687728,0.06872819,-0.01256018...",c8c9c470-c0fd-4379-8382-4da946b6ea69,851f343e-4954-4f0a-8835-9664cc91c181
7,cfb050a5-59a5-4b3f-abe5-c11966c3458f,"[-0.2718665,-0.025269298,0.2906865,0.04861047,...",357134c2-4e0e-4e1e-a503-e95c851a1953,851f343e-4954-4f0a-8835-9664cc91c181
8,8529707b-715e-4741-bad8-038a51e0ac1e,"[0.17470676,0.20757003,0.06747491,-0.017177239...",4389ddfd-c5a6-4e2e-99e1-2de8d04b2a5b,851f343e-4954-4f0a-8835-9664cc91c181
9,fd4b828e-9c1d-49c3-8d80-0aa5b942e10f,"[-0.0264364,0.07242659,-0.029459603,-0.0102718...",c8c9c470-c0fd-4379-8382-4da946b6ea69,851f343e-4954-4f0a-8835-9664cc91c181


In [17]:
pd.read_sql('select * from speakers.speakers',engine)

,id,client_id
0,9da16d6d-5710-44e7-aa1d-ce9cb7221b29,851f343e-4954-4f0a-8835-9664cc91c181
1,357134c2-4e0e-4e1e-a503-e95c851a1953,851f343e-4954-4f0a-8835-9664cc91c181
2,81ddc4cf-0fa2-4702-8f1e-c1eedb2a701c,851f343e-4954-4f0a-8835-9664cc91c181
3,079cc4b7-8060-4545-b2cc-bc118b86b655,851f343e-4954-4f0a-8835-9664cc91c181
4,1e64ed21-1aac-4404-aa5a-c8f5cd13fa9c,851f343e-4954-4f0a-8835-9664cc91c181
5,960931bd-28ef-47d2-b5f9-da652b41db49,851f343e-4954-4f0a-8835-9664cc91c181
6,c8c9c470-c0fd-4379-8382-4da946b6ea69,851f343e-4954-4f0a-8835-9664cc91c181
7,4389ddfd-c5a6-4e2e-99e1-2de8d04b2a5b,851f343e-4954-4f0a-8835-9664cc91c181
8,d8f5b7fe-2745-4017-8f8b-899354e6cae6,851f343e-4954-4f0a-8835-9664cc91c181
9,460bd290-3a6d-4f96-9ea7-797c2c10162b,851f343e-4954-4f0a-8835-9664cc91c181
